In [ ]:
pip install torch==1.7.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 6.6MB 58.6MB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.7.0+cu101 which is incompatible.
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [ ]:
!pip install simpletransformers

Import Data set


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/TUGAS NLP/Sequential labeling/ner_dataset.csv', encoding='latin1')

In [ ]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


mengganti NaN value menjadi sentence 1, sentence 2 dst

In [ ]:
data = data.fillna(method = 'ffill')

In [ ]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


melakukan encoding dengan mentransformasikan ke dalam nilai random

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])

In [ ]:
data.head()

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X = data[["sentence_id","words"]]
Y = data["labels"]

membangun data train dan data test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

membangun train dan test data

In [ ]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words" : x_train["words"],"labels" : y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words" : x_test["words"],"labels" : y_test})

In [ ]:
train_data

,sentence_id,words,labels
789713,29000,but,O
309458,4642,Chavez,I-PER
508485,14719,against,O
896614,34386,ousted,O
392531,8830,Marine,O
...,...,...,...
536998,16189,mishandled,O
653292,22064,denounced,O
34374,6024,'s,O
176704,45828,they,O


Model Training

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
label = data['labels'].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

Percobaan dengan akurasi yang terbaik

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.max_seq_length = 32

In [ ]:
Ketika mulai mengeksekusi model, kata yang sudah di encoding kemudian menjadi masukan model

In [ ]:
model = NERModel('bert','bert-base-cased', labels=label, args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model.train_model(train_data, eval_data = test_data, acc = accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1499, 0.19244468694253314)

Testing

In [ ]:
result,model_output, preds_list = model.eval_model(test_data)

  0%|          | 0/46710 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5839 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.1726080207285319,
 'f1_score': 0.7907241142933574,
 'precision': 0.825692237857467,
 'recall': 0.7585974592583088}

Hasil Uji coba dengan kalimat What is on The Table

In [ ]:
prediction, model_output = model.predict(["what is in the table ?"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'what': 'O'},
  {'is': 'O'},
  {'in': 'O'},
  {'the': 'O'},
  {'table': 'O'},
  {'?': 'O'}]]